# How to use vectorDB class

## Initialize db

In [1]:
from database.vectorDB import VectorDB
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')
password = os.getenv('PASSWORD')
dbname = os.getenv('DB1')

db_config = {
    'user': user,
    'password': password,
    'host': host,
    'port': port,
    'dbname': dbname
}


db = VectorDB(**db_config)


## Create table

In [2]:
table_schema = {
    "id": "UUID PRIMARY KEY",
    "reportId": "INT NOT NULL",  # Assuming 'metadata' table has 'reportId' as a primary key
    "text": "TEXT",
    "embedding": "VECTOR(1536)"  # 1536 dimensions for OpenAI embeddings (adjust as needed)
}

db.create_table("chunkEmbedding", columns=table_schema)

## Insert embedding into table

In [4]:
from langchain.embeddings import OpenAIEmbeddings

openai_api = os.getenv('OPENAI_API_KEY')
text = 'Các giấy tờ có giá đi vay, giấy tờ có giá của khách hàng đưa chiết khấu, tái chiết khấu đã chuyển quyền sở hữu đem đi sử dụng'
embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api)
embeddings = embedder.embed_documents(texts = text)
embedding = embeddings[0]

db.insert_embedding(
    table_name="chunkEmbedding",
    reportId=2,
    text=text,
    Embedding=embedding
)
print("Record inserted successfully.")


Record inserted successfully.


## Query from vector DB

Distance
- L2 - L2 distance
- inner - (negative) inner product
- cosine - cosine distance
- L1 - L1 distance (added in 0.7.0)
- hamming - Hamming distance (binary vectors, added in 0.7.0)
- jaccard - Jaccard distance (binary vectors, added in 0.7.0)

In [6]:
from langchain.embeddings import OpenAIEmbeddings

openai_api = os.getenv('OPENAI_API_KEY')
embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api)

query_text = "công cụ tài chính"

# Query with L2 distance
results = db.query_with_distance(
    query_text, 
    embedder, 
    table_name="chunkEmbedding", 
    distance='L2',
    columns=["id", "reportId", "text"], 
    limit=5
)

print(results)

# Query with cosing distance
results = db.query_with_distance(
    query_text, 
    embedder, 
    table_name="chunkEmbedding", 
    distance='cosine',
    columns=["id", "reportId", "text"], 
    limit=5
)

print(results)

[{'id': 'a3c965a6-8347-42ef-a379-5f43ac5032a1', 'reportId': 1, 'text': 'Các công cụ tài chính phái sinh và các tài sản tài chính khác', 'distance': 1.2224430471154297}, {'id': 'f6fbae66-e107-4b24-8f80-6f5a7d97efae', 'reportId': 1, 'text': 'Các công cụ tài chính phái sinh và các tài sản tài chính khác', 'distance': 1.2224430471154297}, {'id': 'c2f7925d-78f2-49f1-8174-0d4e6a96204d', 'reportId': 2, 'text': 'Các giấy tờ có giá đi vay, giấy tờ có giá của khách hàng đưa chiết khấu, tái chiết khấu đã chuyển quyền sở hữu đem đi sử dụng', 'distance': 1.2224430471154297}]
[{'id': 'a3c965a6-8347-42ef-a379-5f43ac5032a1', 'reportId': 1, 'text': 'Các công cụ tài chính phái sinh và các tài sản tài chính khác', 'distance': 0.7471834270467417}, {'id': 'f6fbae66-e107-4b24-8f80-6f5a7d97efae', 'reportId': 1, 'text': 'Các công cụ tài chính phái sinh và các tài sản tài chính khác', 'distance': 0.7471834270467417}, {'id': 'c2f7925d-78f2-49f1-8174-0d4e6a96204d', 'reportId': 2, 'text': 'Các giấy tờ có giá đi v